In [1]:
# Read the general DTS and save the taxonomy schema to PostgreSQL
from hibachiPostgreSQL import ReadAllGeneralDTSPathesInPostgre # For getting file general DTS pathes
from hibachixbrl import HibachiXmlElement, HibachiETtree
import pandas as pd
import sys
import os

In [2]:
# Get all the general DTS file pathes saved in PostgreSQL 
df_DTSPathes = ReadAllGeneralDTSPathesInPostgre.generalDTSPathes()

Connecting to the PostgreSQL database...
Connection is closed.


In [3]:
def readXMLinGeneralDTS(df_generalDTS):
    ''' Get all the Linkbase, FootnoteLink, Roletype classes in xml file.'''

    # Dataframes for TaxonomySchema, Linkbase, RoleTypes
    df_Linkbases      = pd.DataFrame(columns=['filepath','linkbaseObj'])
    df_RoleTypes      = pd.DataFrame(columns=['filepath','roletypeObj'])       
    
    for CurrentPathName in df_generalDTS['filepath']:
        #Get the extention of the current file
        CurrentFileName = os.path.split(CurrentPathName)[1]
        CurrentFileExtension = os.path.splitext(CurrentPathName)[1] 
        if CurrentFileExtension == '.xml':
            
            try:
                root_ElementTree = HibachiETtree.etree_parse_remove_NS(CurrentPathName)
                if root_ElementTree != None:
                    
                    # List of tags for taxonomy schema, linkbase and roleType/Refs
                    linkbase_tags = ['linkbase','schema']
                    search_tags = linkbase_tags
                    
                    # Search elements with the tags listed above and corresponding create instances
                    for empty_Var, search_tagName in enumerate(search_tags):
                        for schema_ElementTree in root_ElementTree.iter(search_tagName):
                            
                            # 5.1.2 <linkbase>
                            if search_tagName == 'linkbase':
                                df_Linkbases = df_Linkbases.append(\
                                    {'filepath':CurrentPathName,\
                                     'linkbaseObj':HibachiXmlElement.Linkbase(CurrentFileName, schema_ElementTree)}\
                                    ,ignore_index=True
                                )
                                
                            # 5.1.3 <roleType>, 5.1.4 <arcroleType>                     
#                             5.1.3 Defining custom role types - the <roleType> element
#                             The <roleType> element MUST be located among the set of nodes identified by the [XPath 1.0] path "//xsd:schema/xsd:annotation/xsd:appinfo/*".
#                             The value of the @roleURI attribute identifies the @xlink:role attribute value that is being defined.
#                             5.1.4 Defining custom arc role types - the arcroleType element
#                             The <arcroleType> element MUST be among the set of nodes identified by the [XPath 1.0] path "//xsd:schema/xsd:annotation/xsd:appinfo/*".
#                             The value of the @arcroleURI identifies the @xlink:arcrole attribute value that is being defined.

                            elif search_tagName == 'schema':
                                print('schema')
                                for annotation_ElementTree in schema_ElementTree.findall('annotation'):
                                    for appinfo_ElementTree in annotation_ElementTree.findall('appinfo'):
                                        
                                        roleType_Tags = ['roleType','arcroleType']
                                        for empty_Var, temp_tagName in enumerate(roleType_Tags):
                                            for temp_roleType_ElementTree in appinfo_ElementTree.findall(temp_tagName): 
                                                print(temp_tagName)
#                                                 df_RoleTypes = df_RoleTypes.append(\
#                                                     {'filepath':CurrentPathName,\
#                                                      'roletypeObj':HibachiXmlElement.Roletype(CurrentFileName, temp_roleType_ElementTree)}\
#                                                     ,ignore_index=True
#                                                 )
                                                
                                        roleRef_Tags = ['roleRef', 'arcroleRef']
                                        for empty_Var, temp_tagName in enumerate(roleRef_Tags):
                                            for temp_roleRef_ElementTree in appinfo_ElementTree.findall(temp_tagName): 
                                                print(temp_tagName)
#                                                 df_TaxonomySchema = df_TaxonomySchema.append(\
#                                                     {'filepath':CurrentPathName,\
#                                                      'TaxonomySchemaObj':HibachiXmlElement.TaxonomySchema(CurrentFileName, schema_ElementTree)}\
#                                                     ,ignore_index=True
#                                                 )
                                        
                else:
                    print('No root element')

            except:
                print ('Error in reading:', CurrentPathName, sys.exc_info()[0], sys.exc_info()[1])
    return df_Linkbases, df_RoleTypes

In [4]:
# Run the function (this takes a while)
df_Linkbase, df_RoleType = readXMLinGeneralDTS(df_generalDTS=df_DTSPathes)

In [5]:
# Unfold the dataframe to save it in SQL
df_LinkbaseToSQL = pd.concat([df_Linkbase['filepath'],df_Linkbase['linkbaseObj'].apply(lambda x: pd.Series(x.__dict__))],axis=1)

In [124]:
#Add a column containing assined index
#df_LinkbaseToSQL['indexNumber']=df_LinkbaseToSQL.index

In [6]:
#df_LinkbaseToSQL[df_LinkbaseToSQL['childElementList'].apply(lambda x: len(x))>1]['childElementList']
df_LinkbaseToSQL['childElementList']

0                                                       []
1        [{'tag': 'labelLink', 'type': 'extended', 'id'...
2        [{'tag': 'labelLink', 'type': 'extended', 'id'...
3        [{'tag': 'presentationLink', 'type': 'extended...
4        [{'tag': 'presentationLink', 'type': 'extended...
                               ...                        
22871    [{'tag': 'labelLink', 'type': 'extended', 'id'...
22872    [{'tag': 'labelLink', 'type': 'extended', 'id'...
22873    [{'tag': 'presentationLink', 'type': 'extended...
22874    [{'tag': 'presentationLink', 'type': 'extended...
22875    [{'tag': 'referenceLink', 'type': 'extended', ...
Name: childElementList, Length: 22876, dtype: object

In [7]:
# Unfold the LinkbasetoSQL
df_LinkbaseChildElements = df_LinkbaseToSQL.explode('childElementList')[['filepath','childElementList']]
# Format it as dataframe
df_LinkbaseChildElementsToSQL = pd.concat([df_LinkbaseChildElements['filepath'],df_LinkbaseChildElements['childElementList'].apply(lambda x: pd.Series(x))],axis=1)

,filepath,childElementList
0,EdinetGeneraltaxonomy\jpcrp\2013-08-31\label/j...,NaN
1,EdinetGeneraltaxonomy\jpcrp\2013-08-31\label/j...,"{'tag': 'labelLink', 'type': 'extended', 'id':..."
2,EdinetGeneraltaxonomy\jpcrp\2013-08-31\label/j...,"{'tag': 'labelLink', 'type': 'extended', 'id':..."
3,EdinetGeneraltaxonomy\jpcrp\2013-08-31\r/jpcrp...,"{'tag': 'presentationLink', 'type': 'extended'..."
4,EdinetGeneraltaxonomy\jpcrp\2013-08-31\r/jpcrp...,"{'tag': 'presentationLink', 'type': 'extended'..."
...,...,...
22871,EdinetGeneraltaxonomy\jptoo-wto\2013-08-31\lab...,"{'tag': 'labelLink', 'type': 'extended', 'id':..."
22872,EdinetGeneraltaxonomy\jptoo-wto\2013-08-31\lab...,"{'tag': 'labelLink', 'type': 'extended', 'id':..."
22873,EdinetGeneraltaxonomy\jptoo-wto\2013-08-31\r/j...,"{'tag': 'presentationLink', 'type': 'extended'..."
22874,EdinetGeneraltaxonomy\jptoo-wto\2013-08-31\r/j...,"{'tag': 'presentationLink', 'type': 'extended'..."


In [68]:
# Unfold the LinkbaseChildElementsToSQL
df_LinkbaseGrandChildElements = df_LinkbaseChildElementsToSQL.explode('childList')[['filepath','childList']]
# Format it as daraframe
df_LinkbaseGrandChildElementsToSQL = pd.concat([df_LinkbaseGrandChildElements.loc[1]['filepath'],df_LinkbaseGrandChildElements.loc[1]['childList'].apply(lambda x: pd.Series(x))],axis=1)

In [71]:
df_LinkbaseGrandChildElementsToSQL[df_LinkbaseGrandChildElementsToSQL.duplicated()]
#df_LinkbaseGrandChildElements[df_LinkbaseGrandChildElements.duplicated()]
#[df_LinkbaseGrandChildElements.loc[1].duplicated('childList')]
# df_LinkbaseGrandChildElementsToSQL
#df_LinkbaseChildElementsToSQL.loc[1]['childList'][0:2]

,filepath,tag,type,role,arcrole,label,lang,from,to,preferredLabel,weight,order,originalPath,content,use,priority,fragment_ID,searchablePath
